In [158]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import re

import time
import pandas as pd

!which chromedriver

/usr/local/bin/chromedriver


In [159]:
csvpath = pd.read_csv('Resources/mvp_list.csv')
csvpath.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Per Game,Per Game.1,Per Game.2,Per Game.3,Per Game.4,Per Game.5,Shooting,Shooting.1,Shooting.2,Advanced,Advanced.1
0,Season,Lg,Player,Voting,Age,Tm,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
1,2018-19,NBA,Giannis Antetokounmpo\antetgi01,(V),24,MIL,72,32.8,27.7,12.5,5.9,1.3,1.5,.578,.256,.729,14.4,.292
2,2017-18,NBA,James Harden\hardeja01,(V),28,HOU,72,35.4,30.4,5.4,8.8,1.8,0.7,.449,.367,.858,15.4,.289
3,2016-17,NBA,Russell Westbrook\westbru01,(V),28,OKC,81,34.6,31.6,10.7,10.4,1.6,0.4,.425,.343,.845,13.1,.224
4,2015-16,NBA,Stephen Curry\curryst01,(V),27,GSW,79,34.2,30.1,5.4,6.7,2.1,0.2,.504,.454,.908,17.9,.318


In [160]:
mvp_list = csvpath.iloc[1:-1,2].to_list()
mvp_list

['Giannis Antetokounmpo\\antetgi01',
 'James Harden\\hardeja01',
 'Russell Westbrook\\westbru01',
 'Stephen Curry\\curryst01',
 'Stephen Curry\\curryst01',
 'Kevin Durant\\duranke01',
 'LeBron James\\jamesle01',
 'LeBron James\\jamesle01',
 'Derrick Rose\\rosede01',
 'LeBron James\\jamesle01',
 'LeBron James\\jamesle01',
 'Kobe Bryant\\bryanko01',
 'Dirk Nowitzki\\nowitdi01',
 'Steve Nash\\nashst01',
 'Steve Nash\\nashst01',
 'Kevin Garnett\\garneke01',
 'Tim Duncan\\duncati01',
 'Tim Duncan\\duncati01',
 'Allen Iverson\\iversal01',
 "Shaquille O'Neal\\onealsh01",
 'Karl Malone\\malonka01',
 'Michael Jordan\\jordami01',
 'Karl Malone\\malonka01',
 'Michael Jordan\\jordami01',
 'David Robinson\\robinda01',
 'Hakeem Olajuwon\\olajuha01',
 'Charles Barkley\\barklch01',
 'Michael Jordan\\jordami01',
 'Michael Jordan\\jordami01',
 'Magic Johnson\\johnsma02']

In [161]:
master_mvp = []
for player in mvp_list:
    master_mvp.append(player.split('\\')[0])

In [162]:
master_mvp

['Giannis Antetokounmpo',
 'James Harden',
 'Russell Westbrook',
 'Stephen Curry',
 'Stephen Curry',
 'Kevin Durant',
 'LeBron James',
 'LeBron James',
 'Derrick Rose',
 'LeBron James',
 'LeBron James',
 'Kobe Bryant',
 'Dirk Nowitzki',
 'Steve Nash',
 'Steve Nash',
 'Kevin Garnett',
 'Tim Duncan',
 'Tim Duncan',
 'Allen Iverson',
 "Shaquille O'Neal",
 'Karl Malone',
 'Michael Jordan',
 'Karl Malone',
 'Michael Jordan',
 'David Robinson',
 'Hakeem Olajuwon',
 'Charles Barkley',
 'Michael Jordan',
 'Michael Jordan',
 'Magic Johnson']

In [163]:
master_mvp = list(dict.fromkeys(master_mvp))
master_mvp

['Giannis Antetokounmpo',
 'James Harden',
 'Russell Westbrook',
 'Stephen Curry',
 'Kevin Durant',
 'LeBron James',
 'Derrick Rose',
 'Kobe Bryant',
 'Dirk Nowitzki',
 'Steve Nash',
 'Kevin Garnett',
 'Tim Duncan',
 'Allen Iverson',
 "Shaquille O'Neal",
 'Karl Malone',
 'Michael Jordan',
 'David Robinson',
 'Hakeem Olajuwon',
 'Charles Barkley',
 'Magic Johnson']

In [164]:
#Using Jaylen as a proxy to work-around the error
master_mvp = ['Jaylen Brown','Giannis Antetokounmpo','James Harden','Russell Westbrook','Stephen Curry',
              'Kevin Durant','LeBron James','Derrick Rose','Kobe Bryant','Dirk Nowitzki','Steve Nash',
              'Kevin Garnett','Tim Duncan','Allen Iverson',"Shaquille O'Neal",'Karl Malone','Michael Jordan',
              'David Robinson','Hakeem Olajuwon','Charles Barkley','Magic Johnson']

In [165]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.basketball-reference.com/'
browser.visit(url)

In [166]:
player_totals = pd.DataFrame()

for player in master_mvp:
    try:
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        browser.find_by_css('.ac-input').first.fill(player)
        browser.find_by_value('Search').first.click()
        if browser.find_link_by_partial_text(player + " ("):
            browser.click_link_by_partial_text(player + " (")
            
            time.sleep(1)

            table = soup.find('table', {'id':'totals'})
            df = pd.read_html(str(table))[0]
            df['Player'] = player

            player_totals = player_totals.append(df, ignore_index = True)
        else:
            time.sleep(1)

            table = soup.find('table', {'id':'totals'})
            df = pd.read_html(str(table))[0]
            df['Player'] = player

            player_totals = player_totals.append(df, ignore_index = True)
    except:
        pass

In [167]:
player_totals

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player
0,2016-17,20.0,BOS,NBA,SF,78.0,20.0,1341.0,192.0,423.0,...,45.0,175.0,220.0,64.0,35.0,18.0,68.0,142.0,515.0,Giannis Antetokounmpo
1,2017-18,21.0,BOS,NBA,SG,70.0,70.0,2152.0,373.0,803.0,...,66.0,280.0,346.0,114.0,70.0,26.0,124.0,181.0,1017.0,Giannis Antetokounmpo
2,2018-19,22.0,BOS,NBA,SG,74.0,25.0,1913.0,368.0,792.0,...,65.0,248.0,313.0,100.0,69.0,32.0,99.0,186.0,964.0,Giannis Antetokounmpo
3,2019-20,23.0,BOS,NBA,SG,3.0,3.0,86.0,20.0,40.0,...,1.0,19.0,20.0,6.0,2.0,3.0,4.0,11.0,52.0,Giannis Antetokounmpo
4,Career,NaN,NaN,NBA,NaN,225.0,118.0,5492.0,953.0,2058.0,...,177.0,722.0,899.0,284.0,176.0,79.0,295.0,520.0,2548.0,Giannis Antetokounmpo
5,2013-14,19.0,MIL,NBA,SF,77.0,23.0,1897.0,173.0,418.0,...,78.0,261.0,339.0,150.0,60.0,61.0,122.0,173.0,525.0,James Harden
6,2014-15,20.0,MIL,NBA,SG,81.0,71.0,2541.0,383.0,780.0,...,100.0,442.0,542.0,207.0,73.0,85.0,173.0,254.0,1030.0,James Harden
7,2015-16,21.0,MIL,NBA,PG,80.0,79.0,2823.0,513.0,1013.0,...,113.0,499.0,612.0,345.0,94.0,113.0,208.0,258.0,1350.0,James Harden
8,2016-17,22.0,MIL,NBA,SF,80.0,80.0,2845.0,656.0,1259.0,...,142.0,558.0,700.0,434.0,131.0,151.0,234.0,246.0,1832.0,James Harden
9,2017-18,23.0,MIL,NBA,PF,75.0,75.0,2756.0,742.0,1402.0,...,156.0,597.0,753.0,361.0,109.0,106.0,223.0,231.0,2014.0,James Harden


In [168]:
player_totals.to_csv('Outputs/MVP_stats.csv', encoding='utf-8')

In [173]:
player_totals['Player'].value_counts()

Karl Malone              29
Tim Duncan               28
Shaquille O'Neal         26
Michael Jordan           23
Kevin Garnett            23
Charles Barkley          22
Derrick Rose             22
Steve Nash               22
Dirk Nowitzki            21
Magic Johnson            21
Kobe Bryant              20
Allen Iverson            20
David Robinson           19
LeBron James             16
Stephen Curry            16
Hakeem Olajuwon          15
Russell Westbrook        15
Kevin Durant             12
James Harden              8
Giannis Antetokounmpo     5
Name: Player, dtype: int64